In [ ]:
import pandas as pd
import numpy as np
import os
import time
from openai import OpenAI
from langchain.prompts import PromptTemplate

In [ ]:
os.environ["OPENAI_API_KEY"] = "" #Add your OpenAI API 

### Load Tests 

In [ ]:
df_sds = pd.read_csv('SDS.csv', encoding='latin-1')
df_phq = pd.read_csv('PHQ-9.csv', encoding ='latin-1')

### Functions

In [ ]:
#convert dictionary into dataframe
def makeDF(dictionary):
    df_generated = pd.DataFrame({'Questions': list(dictionary.keys()), 'Responses': list(dictionary.values())})
    return df_generated

In [ ]:
#Save dataframe as csv
def save_csv(df, name):
    # Specify the CSV file name
    csv_file = f'{name}.csv'
    # Save the DataFrame to a CSV file
    df.to_csv(csv_file, index=False)

## PHQ-9 and SDS

### Zero-shot

In [ ]:
def GPT(prompt):
    
    client = OpenAI()
    completion = client.chat.completions.create(
      model="gpt-4",
        # model = model,
      messages=[
        # {"role": "system", "content": "You are an assistant"},
        {"role": "user", "content": prompt}
      ],



    )

    return completion.choices[0].message.content
    

In [ ]:
#PHQ-9 Prommpts
# prompt_template1 = PromptTemplate.from_template(
#    '''  Response to this situation: ({situation}) by choosing one of the following options that describe your experience in the past 2 weeks by choosing only one option: 
#    [Not At All - Several Days - More than Half the Days - Nearly Every Day]. 
#    Imagine you are a depression patient. Your answer is '''
# )
# prompt_template2 = PromptTemplate.from_template(
#     '''  Over the past 2 weeks, how often have you been bothered by any of the following problems? The situation is: {situation}. 
#     Response by choosing one of the following options: [Not At All - Several Days - More than Half the Days - Nearly Every Day]. 
#     Imagine you are a depression patient. Your answer is ''' )
# prompt_template3 = PromptTemplate.from_template(
#     '''  Answer this questionnaire that represents you. 
#     how often have you been bothered by any of the following problems? The situation is: {situation}. 
#     Response by choosing one of the following options: [Not At All - Several Days - More than Half the Days - Nearly Every Day]. 
#     Imagine you are a depression patient. Your answer is ''' )

In [ ]:
#SDS prompts
prompt_template1 = PromptTemplate.from_template(" Imagine you are a human. Which best describes how often you felt or behaved this way during the past several days: ({situation}) by choosing only one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Give me only one option. Imagine you are a well-being person. Your answer is ")
prompt_template2 = PromptTemplate.from_template("  Over the past several days, how frequently have you encountered any of the following situations? The situation is: {situation}. Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Imagine you are a well-being person. Your answer is " )
prompt_template3 = PromptTemplate.from_template("How often have you been bothered or experienced by any of the following problems throughout the past several days? The situation is: {situation}. Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Imagine you are a well-being person. Your answer is " )

prompt_depression_template1 = PromptTemplate.from_template(" Imagine you are a depressed human. Which best describes how often you felt or behaved this way during the past several days: ({situation}) by choosing only one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Give me only one option. Imagine you are a depressive disorder patient. Your answer is ")
prompt_depression_template2 = PromptTemplate.from_template(" Imagine you are a depressed human. Over the past several days, how frequently have you encountered any of the following situations? The situation is: {situation}. Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Imagine you are a depressive disorder patient. Your answer is " )
prompt_depression_template3 = PromptTemplate.from_template("Imagine you are a depressed human. How often have you been bothered or experienced by any of the following problems throughout the past several days? The situation is: {situation}. Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. Imagine you are a depressive disorder patient. Your answer is " )


In [ ]:
answer = {}
for i in range(len(df_sds)):
    q = prompt_template3.format(situation=df_sds.iloc[i].Question) #Change the test PHQ-9 OR SDS based on the test required.
    a = GPT(q)
    answer[q] = a



In [ ]:
len(answer)

In [ ]:
answer_df = makeDF(answer)
save_csv(answer_df, "data/tests/SDS/Openai-GPT-4-SDS-baseline-temp_default_trial_3")

### Analysis


In [ ]:
d = pd.read_csv('data/tests/SDS/Openai-GPT-4-SDS-baseline-temp_default_trial_3.csv', encoding='latin-1')
d['Responses'].value_counts()

In [ ]:
import os

# Specify the directory path
folder_path = "data/tests/SDS/"
dd = {}
# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the path is a file (not a directory)
    if os.path.isfile(os.path.join(folder_path, filename)):
        # Print the filename
        data = pd.read_csv(f"data/tests/SDS/{filename}", encoding='latin-1')
        dd[filename] = data['Responses'].value_counts()


In [ ]:
d = makeDF(dd)
save_csv(d,'data/tests/SDS/all_data')

In [ ]:
a = pd.read_csv('data/tests/SDS/all_data.csv', encoding='latin-1')
for index, row in a.iterrows():
    print(row['Questions'], row['Responses'])

In [ ]:
aa = data['Responses'].value_counts()
type(aa)

### Gemini pro

In [ ]:
def count_empty_values(dictionary):
    count = 0
    for value in dictionary.values():
        if value == "":
            count += 1
    return count




In [ ]:
import  google.generativeai as genai

In [ ]:
os.environ["GOOGLE_API_KEY"] = "" #Add your Google Gemini API
genai.configure(api_key="")  #Add your Google Gemini API

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
def gemini(prompt):
    answer = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.0))
    # answer = model.generate_content(prompt)
    # answer = model.generate_content(prompt)

    return answer.text

In [ ]:
def check_and_update(dictionary):
    for key, value in dictionary.items():
        if value == "":
            dictionary[key] = gemini(key)
        time.sleep(15)
    return dictionary

In [ ]:
answer = {}
for i in range(len(df_sds)):
    q = prompt_depression_template1.format(situation=df_sds.iloc[i].Question)
    answer[q] = ''
    

In [ ]:
print(count_empty_values(answer))

In [ ]:
k =  list(answer.keys())
v = list(answer.values())

In [ ]:
# index = 15
# v[index] = gemini(k[index])

In [ ]:
empty_indices = []
for idx, value in enumerate(answer.values()):
    if value == '':
        empty_indices.append(idx)
print(empty_indices)

In [ ]:
answer  = dict(zip(k,v))

In [ ]:
for i in  [19]:
    index = i
    v[index] = gemini(k[index])
    time.sleep(15)

In [ ]:
answer_df = makeDF(answer)
save_csv(answer_df, "data/tests/SDS/Gemini-SDS-depression-temp_0_trial_1")

In [ ]:
# # Run the loop until there are no more empty values
# while any(value == '' for value in answer.values()):
#     your_dictionary = check_and_update(answer)
    



In [ ]:
gemini('''how often have you been bothered or experienced by any of the following problems throughout the past several days? The situation is: I have trouble sleeping at night.. 
    Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. 
    Imagine you are a well-being person. Your answer is ''')

In [ ]:
a =''' '  how often have you been bothered or experienced by any of the following problems throughout the past several days? The situation is: I still enjoy sex.. \n    Response by choosing one of the following options: [A Little of the time - Some of the time - Good Part of the time - Most of the time]. \n    Imagine you are a well-being person. Your answer is ' '''

In [ ]:
'a' in answer.keys()

In [ ]:
answer  = dict(zip(l,v))

In [ ]:
answer[l[6]]

In [ ]:
a in list(answer.keys())

In [ ]:
answer_df = makeDF(answer)
save_csv(answer_df, "data/tests/SDS/Gemini-SDS-baseline-temp_0_trial_1_newtest")